In [9]:
import pandas as pd
import numpy as np

In [10]:
traffic = pd.read_csv("../dataset/last_data/accident_0_min_before.csv", encoding='cp949')
traffic_5min = pd.read_csv("../dataset/last_data/accident_5_min_before.csv", encoding='cp949')
traffic_10min = pd.read_csv("../dataset/last_data/accident_10_min_before.csv", encoding='cp949')
traffic_15min = pd.read_csv("../dataset/last_data/accident_15_min_before.csv", encoding='cp949')


non_traffic = pd.read_csv("../dataset/last_data/non_accident_0_min_before.csv", encoding='cp949')
non_traffic_5min = pd.read_csv("../dataset/last_data/non_accident_5_min_before.csv", encoding='cp949')
non_traffic_10min = pd.read_csv("../dataset/last_data/non_accident_10_min_before.csv", encoding='cp949')
non_traffic_15min = pd.read_csv("../dataset/last_data/non_accident_15_min_before.csv", encoding='cp949')

In [11]:
# non_traffic에서 랜덤으로 25136개 추출
non_traffic = non_traffic.sample(n=25136, random_state=42)
non_traffic_5min = non_traffic_5min.sample(n=25136, random_state=42)
non_traffic_10min = non_traffic_10min.sample(n=25136, random_state=42)
non_traffic_15min = non_traffic_15min.sample(n=25136, random_state=42)

In [12]:
static_traffic = traffic[['lanes', 'speed_limit', 'length', 'bump', 'camera',]]
static_non_traffic = non_traffic[['lanes', 'speed_limit', 'length', 'bump', 'camera',]]

In [13]:
drop_traffic_data = traffic.drop(['datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera' ,'label'], axis=1)
drop_non_traffic_data = non_traffic.drop([ 'datetime', 'link', 'lanes','speed_limit','length','bump', 'camera', 'label'], axis=1)

drop_traffic_data_5 = traffic_5min.drop(['datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)
drop_non_traffic_data_5 = non_traffic_5min.drop([ 'datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera','label'], axis=1)

drop_traffic_data_10 = traffic_10min.drop(['datetime', 'link', 'lanes','speed_limit','length' ,'bump', 'camera','label'], axis=1)
drop_non_traffic_data_10 = non_traffic_10min.drop([ 'datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)

drop_traffic_data_15 = traffic_15min.drop(['datetime', 'link', 'lanes','speed_limit','length', 'bump', 'camera','label'], axis=1)
drop_non_traffic_data_15 = non_traffic_15min.drop(['datetime', 'link', 'lanes','speed_limit','length','bump', 'camera','label'], axis=1)

traffic_label = traffic['label']
non_traffic_label = non_traffic['label']

In [14]:
dynamic_traffic_5 = drop_traffic_data_5.values
dynamic_traffic_10 = drop_traffic_data_10.values
dynamic_traffic_15 = drop_traffic_data_15.values

dynamic_non_traffic_5 = drop_non_traffic_data_5.values
dynamic_non_traffic_10 = drop_non_traffic_data_10.values
dynamic_non_traffic_15 = drop_non_traffic_data_15.values

traffic_label = traffic_label.values
non_traffic_label = non_traffic_label.values

static_traffic = static_traffic.values
static_non_traffic = static_non_traffic.values

In [15]:
accident_data_5min = np.concatenate((dynamic_traffic_5, static_traffic), axis=1)
accident_data_10min = np.concatenate((dynamic_traffic_10, static_traffic), axis=1)
accident_data_15min = np.concatenate((dynamic_traffic_15, static_traffic), axis=1)

non_accident_data_5min = np.concatenate((dynamic_non_traffic_5, static_non_traffic), axis=1)
non_accident_data_10min = np.concatenate((dynamic_non_traffic_10, static_non_traffic), axis=1)
non_accident_data_15min = np.concatenate((dynamic_non_traffic_15, static_non_traffic), axis=1)

In [16]:
from sklearn.model_selection import train_test_split

X, Y = [], []
for accident in range(len(dynamic_traffic_5)):
    X.append([dynamic_traffic_15[accident], dynamic_traffic_10[accident], dynamic_traffic_5[accident]])
    Y.append(traffic_label[accident])
for non_accident in range(len(dynamic_non_traffic_5)):
    X.append([dynamic_non_traffic_15[non_accident], dynamic_non_traffic_10[non_accident], dynamic_non_traffic_5[non_accident]])
    Y.append(non_traffic_label[non_accident])
X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40, shuffle=True)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=40, shuffle=True)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, SimpleRNN
from tensorflow.keras.models import Model


In [20]:
input_data = Input(shape=(X_train.shape[1], X_train.shape[2]))

lstm_1 = LSTM(32, return_sequences=True)(input_data)
lstm_2 = LSTM(32, return_sequences=False)(lstm_1)
dense_1 = Dense(50, activation='relu')(lstm_2)
dense_2 = Dense(1, activation='sigmoid')(dense_1)

lstm_model = Model(inputs=input_data, outputs=dense_2)
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.BinaryAccuracy(threshold=0.5)])

In [21]:
hist = lstm_model.fit(X_train, Y_train, epochs=30, batch_size=32, validation_data=(X_val, Y_val))

loss, acc, auc, precision, recall, binary_accuracy = lstm_model.evaluate(X_test, Y_test, verbose=0)
print("m parameter", lstm_model.count_params())
print('loss : ', loss)
print('acc : ', acc)
print('auc : ', auc)
print('precision : ', precision)
print('recall : ', recall)
print('binary_accuracy : ', binary_accuracy)
#print f1-score with calculate recall precision

f1_score = (2 * precision * recall) / (precision + recall)
print('f1_score : ', f1_score)

Epoch 1/30
1257/1257 [==============================] - 4s 2ms/step - loss: 0.6655 - accuracy: 0.5856 - auc: 0.6240 - precision_1: 0.6016 - recall_1: 0.5104 - binary_accuracy: 0.5856 - val_loss: 0.6646 - val_accuracy: 0.5898 - val_auc: 0.6376 - val_precision_1: 0.6788 - val_recall_1: 0.3462 - val_binary_accuracy: 0.5898
Epoch 2/30
1257/1257 [==============================] - 3s 2ms/step - loss: 0.6579 - accuracy: 0.5944 - auc: 0.6391 - precision_1: 0.6342 - recall_1: 0.4489 - binary_accuracy: 0.5944 - val_loss: 0.6602 - val_accuracy: 0.5900 - val_auc: 0.6355 - val_precision_1: 0.5965 - val_recall_1: 0.5650 - val_binary_accuracy: 0.5900
Epoch 3/30
1257/1257 [==============================] - 3s 2ms/step - loss: 0.6567 - accuracy: 0.5971 - auc: 0.6415 - precision_1: 0.6371 - recall_1: 0.4537 - binary_accuracy: 0.5971 - val_loss: 0.6601 - val_accuracy: 0.5940 - val_auc: 0.6395 - val_precision_1: 0.6347 - val_recall_1: 0.4492 - val_binary_accuracy: 0.5940
Epoch 4/30
1257/1257 [============